In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/semeval2024task8/subtaskA_dev_monolingual.jsonl
/kaggle/input/semeval2024task8/subtaskA_train_monolingual.jsonl
/kaggle/input/semeval2024task8/subtaskA_train_multilingual.jsonl
/kaggle/input/semeval2024task8/subtaskB_train.jsonl
/kaggle/input/semeval2024task8/subtaskA_dev_multilingual.jsonl
/kaggle/input/semeval2024task8/subtaskC_dev.jsonl
/kaggle/input/semeval2024task8/subtaskB_dev.jsonl
/kaggle/input/semeval2024task8/subtaskC_train.jsonl


In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.3 MB/s eta 0:00:00eta 0:00:01


In [3]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification, AdamW, set_seed
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score
from tqdm import tqdm

In [4]:
def get_data(train_path, test_path, random_seed):

    train_df = pd.read_json(train_path, lines=True)
    test_df = pd.read_json(test_path, lines=True)
    
    train_df, val_df = train_test_split(train_df, test_size=0.2, stratify=train_df['label'], random_state=random_seed)

    return train_df, val_df, test_df

In [13]:
random_seed = 0
set_seed(random_seed)
train_path =  '/kaggle/input/semeval2024task8/subtaskA_train_multilingual.jsonl'
test_path =  '/kaggle/input/semeval2024task8/subtaskA_dev_multilingual.jsonl'

train_df, valid_df, eval_df = get_data(train_path, test_path, random_seed)

In [14]:
train_df

,text,label,model,source,id
97884,This paper proposes an approach to learning a ...,0,human,peerread,97884
92020,فیس بک کے بانی مارک زکربرگ نے بلا سوچے سمجھے ک...,0,human,urdu,92020
64939,A DJ mix can be created using any number of di...,1,bloomz,wikihow,64939
55346,Jakarta -\nKapolda Metro Jaya Irjen Idham Azis...,0,human,indonesian,55346
163088,We analyze the relationship between star for...,0,human,arxiv,163088
...,...,...,...,...,...
103298,\nThis paper addresses the problem of predicti...,1,cohere,peerread,103298
128253,If your hair is going to stay straightened an...,0,human,wikihow,128253
12019,The Association of International Schools in A...,1,cohere,wikipedia,12019
163907,Image-guided depth completion aims to genera...,0,human,arxiv,163907


In [15]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [16]:


id2label = {0: "human", 1: "machine"}
label2id = {"human": 0, "machine": 1}
model_name = 'xlm-roberta-base'
# Initialize the tokenizer and model
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=2, id2label=id2label, label2id=label2id).to(device)


Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [44]:
eval_df

,text,label,model,source,id
0,"\n\nЮбилей Академии наук СССР, запланированный...",1,davinci,russian,0
1,\n\nВ Мичигане был окончательно подведен итог ...,1,davinci,russian,1
2,"\n\nРежиссер Джош Транк, известный нам по филь...",1,davinci,russian,2
3,"\n\nПонедельник – день, когда в «Разведке» пыт...",1,davinci,russian,3
4,\n\nДатская арт-группа Surrend провела провока...,1,davinci,russian,4
...,...,...,...,...,...
3995,Zwischenfälle auf dem Nangpa La beziehen sich ...,1,chatGPT,german,3995
3996,Zwischenmenschliche Beziehungen zählen zu den ...,1,chatGPT,german,3996
3997,"Das Zwölftonsystem, auch Dodekaphonie genannt,...",1,chatGPT,german,3997
3998,"Eine Zyklische Aktie ist eine Aktie, die aufgr...",1,chatGPT,german,3998


In [45]:
train_df

,text,label,model,source,id
97884,This paper proposes an approach to learning a ...,0,human,peerread,97884
92020,فیس بک کے بانی مارک زکربرگ نے بلا سوچے سمجھے ک...,0,human,urdu,92020
64939,A DJ mix can be created using any number of di...,1,bloomz,wikihow,64939
55346,Jakarta -\nKapolda Metro Jaya Irjen Idham Azis...,0,human,indonesian,55346
163088,We analyze the relationship between star for...,0,human,arxiv,163088
...,...,...,...,...,...
103298,\nThis paper addresses the problem of predicti...,1,cohere,peerread,103298
128253,If your hair is going to stay straightened an...,0,human,wikihow,128253
12019,The Association of International Schools in A...,1,cohere,wikipedia,12019
163907,Image-guided depth completion aims to genera...,0,human,arxiv,163907


In [53]:
# Tokenize text data from DataFrame
train_encodings = tokenizer(train_df['text'].tolist(), truncation=True, padding=True)
eval_encodings = tokenizer(eval_df['text'].tolist(), truncation=True, padding=True)


In [54]:
class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(device)
        return item

    def __len__(self):
        return len(self.labels)
    


In [55]:
train_dataset = CustomDataset(train_encodings, train_df['label'].tolist())
eval_dataset = CustomDataset(eval_encodings, eval_df['label'].tolist())

In [77]:
for i,data in enumerate(train_dataset):
    print(data)
    if(i>10):
        break

{'input_ids': tensor([     0,   3293,  15122,  26171,      7,    142,  51515,     47,  52080,
            10,    484, 109109,    366,   2189,  17368,    142,     22,    587,
           820,      9,  50718,  51744,    108,  82451,      6, 159958,      4,
           678,     70, 157167,  54700,  60213,    450,     70,    484, 109109,
        140992,     83,   4393,  80866,     41,  10484,      5,    581,  55300,
            83, 151575,    297,    645,   6626,   5570,   2053,   3509,      7,
            15,  10429,     31, 111056,    136,     62,  80091,    247,    237,
          5299,    237,     10,  21261,   2053,   3509,  33444,    214,     47,
         12937,  33938,      5,   3293,     83,     10,  18652,      4,   5299,
         71924,    297,  15122,      4,   3129,  51776,     10,  35845,    538,
          5299, 170920,  61353,     23,     70,   3173,    111,    142,     22,
           587,    820,      9,  50718,  51744,    678,   3060,   1927,  58838,
             7,     15,   

In [59]:
# Define training parameters
batch_size = 16
learning_rate = 2.0e-5
num_epochs = 3

In [60]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=batch_size)

In [63]:

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [67]:
# Function for training
def train(model, optimizer, criterion, dataloader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in tqdm(dataloader, desc=f"Epoch {epoch + 1}/{num_epochs}", leave=False):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            print(loss)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * input_ids.size(0)

        epoch_loss = running_loss / len(dataloader.dataset)
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}")

        
# Function for evaluation
def evaluate(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in tqdm(dataloader, desc="Evaluating"):
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']

            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs.logits, dim=1)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Accuracy: {accuracy:.4f}")

In [78]:
# # Train the baseline model
# print("Training Baseline Model...")
# train(model, optimizer, criterion, train_loader, num_epochs)
print("Evaluating Baseline Model...")
evaluate(model, eval_loader)

Evaluating Baseline Model...


Evaluating: 100%|██████████| 250/250 [02:06<00:00,  1.97it/s]

Accuracy: 0.5030
